# NPM Supply Chain Network Analysis
## Automated Analysis Pipeline

This notebook performs a comprehensive analysis of the NPM supply chain network. It executes the following steps:
1.  **Data Acquisition**: Fetches the most dependent packages from `ecosyste.ms`.
2.  **Network Construction**: Builds a dependency graph using `networkx`.
3.  **Risk Assessment**: Calculates centrality metrics and a composite **Behavioral Risk Score (BRS)**.
4.  **Robustness Simulation**: Simulates targeted and random attacks to evaluate network resilience.
5.  **Visualization**: Generates plots for network topology, degree distributions, and simulation results.


In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import sys
import os

# Add current directory to path to ensure imports work
sys.path.append(os.getcwd())

# Import custom modules
from data_loader import get_top_dependents
from network_builder import build_dependency_graph
from metrics import calculate_risk_scores
from simulation import simulate_attacks, calculate_single_node_impact
from utils import export_results
from visualize import (
    plot_network_structure, 
    plot_degree_distributions, 
    plot_correlations, 
    plot_top_risk_scores, 
    plot_simulation_results,
    plot_risk_vs_cascade
)

# --- Configuration ---
TOP_N = 1000            # Number of seed packages to fetch
MAX_DEPTH = 7           # Traversal depth for the crawler
BETWEENNESS_K = 100     # Sampling size for faster betweenness calculation
API_DELAY = 0.1         # Delay between API calls in seconds
OUTPUT_DIR = '../results'  # Merkezi çıktı dizini (Proje kökünde)

# --- Visualization Setup ---
sns.set_theme(style="whitegrid", context="notebook", palette="deep")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['figure.dpi'] = 100

## 2. Data Acquisition
We fetch the top `TOP_N` packages sorted by the number of dependent repositories from the `ecosyste.ms` API. These packages serve as the "seed" nodes for our network crawler.


In [ ]:
# Fetch seed packages
seed_packages = get_top_dependents(limit=TOP_N, api_delay=API_DELAY)

## 3. Network Construction
Using the seed packages, we build a directed dependency graph. The crawler traverses dependencies up to `MAX_DEPTH`.
- **Nodes**: Packages
- **Edges**: Directed dependencies (Package A depends on Package B)


In [ ]:
# Build Graph
G = build_dependency_graph(seed_packages, max_depth=MAX_DEPTH, api_delay=API_DELAY)

In [ ]:
# Print Graph Statistics
print(f"Graph Statistics:")
print(f" - Nodes: {G.number_of_nodes()}")
print(f" - Edges: {G.number_of_edges()}")
print(f" - Density: {nx.density(G):.6f}")
print(f" - Is Strongly Connected: {nx.is_strongly_connected(G)}")
print(f" - Is Weakly Connected: {nx.is_weakly_connected(G)}")


## 4. Metrics & Risk Scoring
We calculate centrality metrics to identify critical nodes:
- **In-Degree**: Number of packages depending on a node within the network (Popularity in the backbone).
- **Out-Degree**: Number of dependencies a node has (Complexity and propagation potential).
- **Betweenness Centrality**: Frequency of a node appearing on shortest paths (Bridge/Bottleneck role).
- **Dependents Count**: Number of packages depending on this node in the entire NPM ecosystem (Real-world impact).

**Behavioral Risk Score (BRS) Formula:**
$$BRS = 0.25 \times InDegree_{norm} + 0.25 \times OutDegree_{norm} + 0.25 \times Betweenness_{norm} + 0.25 \times DependentsCount_{norm}$$

All four metrics are equally weighted (25% each) to reflect the multifaceted nature of supply chain criticality: network popularity, complexity, structural importance, and ecosystem impact.


In [ ]:
# Calculate Metrics
risk_df = calculate_risk_scores(G, betweenness_k=BETWEENNESS_K)
print("\nTop 10 Critical Packages:")
display(risk_df.head(10))

## 5. Robustness Simulation
We simulate supply chain attacks by removing nodes and measuring the network's fragmentation.
- **Targeted Attack**: Removing nodes with the highest BRS.
- **Random Attack**: Removing random nodes.
- **Metric**: Size of the Largest Connected Component (LCC). A faster decay in LCC indicates higher vulnerability.

We also calculate the **Cascade Impact** for the top risky nodes, defined as the reduction in LCC size when that single node is removed.


In [ ]:
# Run Simulations
print("Running robustness simulations...")
sim_results = simulate_attacks(G, risk_df, num_removals=50)

# Calculate Cascade Impact
print("Calculating cascade impact for top nodes...")
impact_df = calculate_single_node_impact(G, risk_df, sample_size=100)
print("Top 5 Packages by Cascade Impact:")
display(impact_df.head(5))

In [ ]:
# 1. Network Topology
plot_network_structure(G, output_dir=OUTPUT_DIR + '/plots')

# 2. Degree Distributions
plot_degree_distributions(G, output_dir=OUTPUT_DIR + '/plots')

# 3. Correlations (Metrics)
plot_correlations(risk_df, output_dir=OUTPUT_DIR + '/plots')

# 4. Top Risk Scores
plot_top_risk_scores(risk_df, output_dir=OUTPUT_DIR + '/plots')

# 5. Risk vs Cascade Impact Correlation
plot_risk_vs_cascade(impact_df, output_dir=OUTPUT_DIR + '/plots')

# 6. Simulation Results (Robustness)
plot_simulation_results(sim_results, output_dir=OUTPUT_DIR + '/plots')

# --- NEW VISUALIZATIONS: Strategic Insights ---

# 7. Multi-metric Heatmap (Shows complexity of risk assessment)
from visualize import plot_metric_heatmap, plot_brs_components, plot_ecosystem_vs_network, plot_risk_distribution_by_type
from visualize import plot_brs_vs_cascade_validation, plot_brs_distribution, plot_top20_brs_component_lines

plot_metric_heatmap(risk_df, output_dir=OUTPUT_DIR + '/plots')

# 8. BRS Components Breakdown (Shows contribution of each factor)
plot_brs_components(risk_df, output_dir=OUTPUT_DIR + '/plots')

# 9. Ecosystem vs Network Analysis (Backbone vs broader ecosystem)
plot_ecosystem_vs_network(risk_df, output_dir=OUTPUT_DIR + '/plots')

# 10. Risk Distribution by Package Type (Seed vs discovered)
plot_risk_distribution_by_type(risk_df, output_dir=OUTPUT_DIR + '/plots')

# --- VALIDATION PLOTS: Proving BRS Formula Effectiveness ---

# 11. BRS vs Cascade Impact Validation (CRITICAL: Shows BRS predicts real network damage)
plot_brs_vs_cascade_validation(impact_df, output_dir=OUTPUT_DIR + '/plots')

# 12. BRS Distribution Analysis (Shows realistic, power-law characteristics)
plot_brs_distribution(risk_df, output_dir=OUTPUT_DIR + '/plots')

# 13. Top 20 BRS Component Lines Chart (Shows how BRS components align with actual cascade impact)
plot_top20_brs_component_lines(risk_df, impact_df, output_dir=OUTPUT_DIR + '/plots')

# 14. Export Data
export_results(risk_df, G, impact_df=impact_df, output_dir=OUTPUT_DIR)

# 15. Save Graph Object
nx.write_gml(G, OUTPUT_DIR + "/network.gml")
print(f" - Saved {OUTPUT_DIR}/network.gml")

print("\n" + "="*70)
print("ANALYSIS COMPLETE!")
print("="*70)
print(f"All visualizations and validation plots saved to {OUTPUT_DIR}/plots/")
print(f"Total packages analyzed: {len(risk_df)}")
print(f"Network nodes: {G.number_of_nodes()}, edges: {G.number_of_edges()}")
print("Total plots generated: 13 comprehensive visualizations")
print("="*70)

## 6. Visualization & Reporting
We generate comprehensive visualizations to demonstrate the criticality of NPM supply chain packages:

**Core Analysis Visualizations:**
- Network topology and degree distributions
- Metric correlations and top risk scores
- Robustness simulation results (LCC decay)

**Strategic Insight Visualizations:**
- Multi-metric heatmap: Shows the multidimensional nature of package criticality
- BRS components breakdown: Demonstrates how different factors contribute to overall risk
- Ecosystem vs Network analysis: Reveals the relationship between backbone network presence and broader ecosystem impact
- Risk distribution by type: Demonstrates the value of discovering hidden critical packages

**VALIDATION PLOTS: Proving the BRS Formula is Meaningful:**
- **BRS vs Cascade Impact**: The critical validation - demonstrates that packages with high BRS scores actually cause more network damage when removed. High Pearson correlation proves the formula's predictive power.
- **BRS Distribution Analysis**: Shows that BRS scores follow a realistic heavy-tailed (power-law) distribution, typical of real supply chain networks. Includes:
  - Linear histogram: Shows overall distribution shape
  - Log-log plot: Reveals power-law characteristics (α parameter)
  - Q-Q plot: Tests normality (confirms non-normal, realistic distribution)
  - Box plot with statistics: Shows mean, median, and skewness

All results are exported to CSV files and a comprehensive README report in the results directory.


## 7. Conclusion
The analysis highlights the structural vulnerabilities in the NPM supply chain.
- **Critical Nodes**: Packages with high BRS are potential single points of failure.
- **Robustness**: The difference between targeted and random attack simulations demonstrates the network's fragility against intelligent adversaries.
- **Cascade Impact**: Certain nodes, when removed, cause a significant drop in network connectivity, indicating their role as critical bridges.

Future work could involve analyzing the maintainer network or including transitive dependencies in the risk score.
